<a href="https://colab.research.google.com/github/bbogdanovaalina/persistent-laplacian/blob/main/%D0%A1%D1%87%D0%B5%D1%82_%D0%BB%D0%B0%D0%BF%D0%BB%D0%B0%D1%81%D0%B8%D0%B0%D0%BD%D0%BE%D0%B2_%D0%B2%D1%82%D0%BE%D1%80%D0%BE%D0%B9_%D0%B2%D0%B0%D1%80%D0%B8%D0%B0%D0%BD%D1%82.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install dionysus


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 4.9 MB/s 
  Created wheel for dionysus: filename=dionysus-2.0.8-cp37-cp37m-linux_x86_64.whl size=322709 sha256=449f34c245397dc0c0ef97109da3f89b8c73f5b62e3193979dc22d2888388dc2
  Stored in directory: /root/.cache/pip/wheels/2b/bb/90/2e8a30f3af9a13951d8c76fa73689f6b8d87234c603bbd7003
Successfully built dionysus


In [2]:
pip install gudhi


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 29.3 MB 61.9 MB/s 


In [3]:
import numpy as np
import dionysus as d
from itertools import permutations
from numpy import linalg
import matplotlib
import matplotlib.pyplot as plt
import gudhi as gd
from gudhi import SimplexTree
import pandas as pd
import seaborn as sns

In [ ]:
points = np.loadtxt(names_con[2], delimiter = ',', skiprows = 1) 

In [15]:
  # из набора points строим фильтрацию rips_list = список (вложенных комплекс, время рождения)


def data1(file):

    points = np.loadtxt(file, delimiter = ',', skiprows = 1)
    corr_m = np.corrcoef(points, rowvar=False)
    new_corr = 1 - abs(corr_m)
    skeleton = gd.RipsComplex(new_corr)#, max_edge_length = 3) # тут максимальная длина ребра ОЧЕНЬ увеличивает время
    Rips_simplex_tree_sample = skeleton.create_simplex_tree(max_dimension = 5)
    #print(Rips_simplex_tree_sample.dimension(), ' - максимальная размерность симплексов')
    #print(Rips_simplex_tree_sample.num_vertices(), ' - количество вершин')
    #print(Rips_simplex_tree_sample.num_simplices(), ' - количество симплексов')
    rips_generator = Rips_simplex_tree_sample.get_filtration()
    rips_list = list(rips_generator)


    def ind_of_compl(rips_list, dim=5, n=20):                                              
        num_compl = 0
        ans = 0
        i = 0
        while num_compl < n and i != len(rips_list):
          if len(rips_list[i][0]) == dim + 1:
            num_compl += 1
            ans = i
          i += 1
        return ans

    rips_list = rips_list[:ind_of_compl(rips_list, 5, 500) + 1]


    number_of_simplicies = [len([i for i in rips_list if len(i[0])==1]),
                                len([i for i in rips_list if len(i[0])==2]),
                                    len([i for i in rips_list if len(i[0])==3]),
                                        len([i for i in rips_list if len(i[0])==4]),
                                            len([i for i in rips_list if len(i[0])==5]),
                                                len([i for i in rips_list if len(i[0])==6])]
    number_of_simplicies = np.array(number_of_simplicies)
    number_of_vertices = number_of_simplicies[0]
    for i in range(len(number_of_simplicies)):
        print(number_of_simplicies[i], ' - количество комплексов размерности ', i)

    # переделываем симплексы из SimplexTree в gudhi в SimplexTree из dionysus
    d_rips_list = d.Filtration()
    for vertices, time in rips_list:
      d_rips_list.append(d.Simplex(vertices, time))
    d_rips_list.sort()



    # граничный оператор возвращает ориентацию грани i симплекса j
    def boundary_sign(i, j):
        for k in range(len(j) - 1):
          if j[k] != i[k]:
            if k % 2 == 0:
              return 1
            else:
              return -1
          if (len(j) - 1) % 2 == 0:
            return 1
          else:
            return -1

    # проверяет являются ли симплексы i и j (одинаковой размерности) смежными (то есть имеют общую грань на размерность меньше)
    def adjacent(i, j):
      face = set(i.boundary()).intersection(set(j.boundary()))
      if len(face) > 0:
          return True, list(face)[0] 
      else:
        return False, []

    # возвращает дополнение Шура к правой нижней матрице размера n_L - n_K в матрице M 
    def Schur_complement(M, n_K):  # M будет Delta^L_q,up , K - номер вложенного в L комплекса
      n_L = len(M)
      A = M[np.ix_(range(n_K), range(n_K))]
      B = M[np.ix_(range(n_K), range(n_L - n_K, n_L))]
      C = M[np.ix_(range(n_L - n_K, n_L), range(n_K))]
      D = M[np.ix_(range(n_L - n_K, n_L), range(n_L - n_K, n_L))]
      return A - np.dot(np.dot(B, linalg.pinv(D)), C)




    def Fast_laplacian(K, L, q):
      #Delta_q_K_down = D_down(K, q, d_rips_list, weights) - A_down(K, q, d_rips_list, weights)
      #Delta_q_L_up = D_up(L, q, d_rips_list, weights) - A_up(L, q, d_rips_list, weights)
      
      d_rips_list_q_K = d.Filtration()
      for i in range(K):
        if d_rips_list[i].dimension() == q:
          d_rips_list_q_K.append(d_rips_list[i])

      d_rips_list_qm1_K = d.Filtration()
      for i in range(K):
        if d_rips_list[i].dimension() == q + 1:
          d_rips_list_qm1_K.append(d_rips_list[i])

      n_q_K = len(d_rips_list_q_K)

      D_q_K_down = np.zeros(n_q_K)  
      for i in range(n_q_K):
        for sb in d_rips_list_q_K[i].boundary():
          D_q_K_down[i] += 1 # тут вес(d_rips_list_q[i]) / вес(sb) вместо 1, если с весами

      A_q_K_down = np.zeros((n_q_K, n_q_K))  
      for i, j in permutations(d_rips_list_q_K, 2):
        adj, l = adjacent(i, j)
        if adj:
          A_q_K_down[d_rips_list_q_K.index(i), d_rips_list_q_K.index(j)] += - boundary_sign(l, i) * boundary_sign(l, j) * 1# тут вес(i) / вес(l) вместо 1, если с весами
    
      Delta_q_K_down = np.diag(D_q_K_down) - A_q_K_down



      d_rips_list_q_L = d.Filtration()
      for i in range(L):
        if d_rips_list[i].dimension() == q:
          d_rips_list_q_L.append(d_rips_list[i])  

      d_rips_list_qp1_L = d.Filtration()
      for i in range(L):
        if d_rips_list[i].dimension() == q + 1:
          d_rips_list_qp1_L.append(d_rips_list[i])

      n_q_L = len(d_rips_list_q_L)

      A_q_L_up = np.zeros((n_q_L, (n_q_L)))  
      for l in range(len(d_rips_list_qp1_L)):
        for i, j in permutations(d_rips_list_qp1_L[l].boundary(), 2):
          A_q_L_up[d_rips_list_q_L.index(i), d_rips_list_q_L.index(j)] += - boundary_sign(i, d_rips_list_qp1_L[l]) * boundary_sign(j, d_rips_list_qp1_L[l]) * 1# тут вес(d_rips_list_qp1[l]) / вес(j) вместо 1, если с весами

      D_q_L_up = np.zeros(n_q_L)  
      for s in d_rips_list_qp1_L:
        for sb in s.boundary():
          D_q_L_up[d_rips_list_q_L.index(sb)] += 1 # тут вес(s) / вес(sb) вместо 1, если с весами

      Delta_q_L_up = np.diag(D_q_L_up) - A_q_L_up
      
      if n_q_L == n_q_K: 
        return Delta_q_K_down + Delta_q_L_up
      else:
        Delta_q_K_L_up = Schur_complement(Delta_q_L_up, n_q_K)
      return Delta_q_K_down + Delta_q_K_L_up




    res = [] # ответы

    Q = 1
    #p = 0.8
    '''max_L = int(np.sum(number_of_simplicies[:3]) * p)
    for L in (max_L // 2, max_L // 4 * 3, max_L):
      for K in (L // 2, L // 4 * 3, L):'''

    for L in (ind_of_compl(rips_list, Q, 300), ind_of_compl(rips_list, Q, 400), ind_of_compl(rips_list, Q, 500)):
      for K in (L // 2, L):
          res.append( sorted([i.real for i in np.linalg.eigvals(Fast_laplacian(K, L, Q))]) )
            

    Q = 2
    #max_L = int(np.sum(number_of_simplicies[:4]) * p)
    '''for L in (max_L // 2, max_L):
      for K in (L // 2, L):'''
    for L in (ind_of_compl(rips_list, Q, 300), ind_of_compl(rips_list, Q, 400), ind_of_compl(rips_list, Q, 500)):
      for K in (L // 2, L):
        res.append( sorted([i.real for i in np.linalg.eigvals(Fast_laplacian(K, L, Q))])  )
       

    Q = 3
    #max_L = int(np.sum(number_of_simplicies[:5]) * p)
    '''for L in (max_L // 2, max_L):
      for K in (L // 2, L):'''
    for L in (ind_of_compl(rips_list, Q, 300), ind_of_compl(rips_list, Q, 400), ind_of_compl(rips_list, Q, 500)):
      for K in (L // 2, L):
        res.append( sorted([i.real for i in np.linalg.eigvals(Fast_laplacian(K, L, Q))]) )
        


    Q = 4
    #max_L = int(np.sum(number_of_simplicies) * p)
    '''for L in (max_L // 2, max_L):
      for K in (L // 2, L):'''
    for L in (ind_of_compl(rips_list, Q, 450), ind_of_compl(rips_list, Q, 500)):
      for K in (int(L * 0.6), int(L * 0.8), L):
        res.append( sorted([i.real for i in np.linalg.eigvals(Fast_laplacian(K, L, Q))])  )
       


    Q = 5
    '''for L in (max_L // 10 * 9, max_L):
      for K in (L // 4 * 3, L):'''
    for L in (ind_of_compl(rips_list, Q, 450), ind_of_compl(rips_list, Q, 500)):
      for K in (int(L * 0.6), int(L * 0.8), L):
        res.append( sorted([i.real for i in np.linalg.eigvals(Fast_laplacian(K, L, Q))])  )
        

    import csv

    with open(file[:-4] + '_result.csv', "w", newline="") as file:
        writer = csv.writer(file)
        writer.writerows(res)

    return res

In [7]:
names_pat = []
with open('files_patients_MSDL_run_1.txt', 'r') as f:
    file = f.readlines()
    for line in file:
      if 'sub' in line:
        names_pat.append(line[:-1])
f.close()

'''names_con = []
with open('files_con_run_1.txt', 'r') as f:
    file = f.readlines()
    for line in file:
      if 'sub' in line:
        names_con.append(line[:-1])
f.close()'''

"names_con = []\nwith open('files_con_run_1.txt', 'r') as f:\n    file = f.readlines()\n    for line in file:\n      if 'sub' in line:\n        names_con.append(line[:-1])\nf.close()"

In [ ]:
num = 0
n = 0
for name in names_pat[num * 10: num * 10 + 10]:
  data1(name)
  print(n)
  n += 1

In [22]:
from google.colab import files

In [ ]:
for name in names_pat[num * 10: num * 10 + 10]:
    files.download('/content/' + name[:-4] + '_result.csv')